In [1]:
import os
import pandas as pd
import numpy as np
from utils import to_21d_vec, get_beat_vector, specialChords, specialChordsABC
import re
import sys
sys.path.insert(1, './harmalysis')
from inputRN2Chord import inputRN

In [8]:
script_dir = os.getcwd()
score_rel_path = "../datasets/ABC/ABC_reduced_score_for_vec/n11op95_02_score_reduced4_tmp_A4_trans_vector_pre.krn"
scorepath = os.path.join(script_dir, score_rel_path)


In [9]:
df = pd.read_csv(scorepath, sep="\t", header=None)
pd.set_option('display.max_rows', df.shape[0]+1)
#df.columns = ["harm", "root", "voice4", "voice3", "voice2", "voice1", "index", "beat", "measure", "key", "meter" ]
#df.columns = ["harm", "root", "voice4", "voice3", "voice2", "voice1", "beat", "measure", "key", "meter" ]
#df.columns = ["harm", "voice4", "voice3", "voice2", "voice1", "index", "beat", "measure", "key", "meter" ]
#df.columns = ["harm", "voice4", "voice3", "voice2", "voice1", "beat", "measure", "key", "meter" ]

#sear org
#df.columns = ["voice4", "voice3", "voice2", "voice1", "harm", "beat", "measure", "key", "meter" ]

# abc org
#df.columns = ["voice4", "dy4", "voice3", "dy3", "voice2", "dy2", "voice1", "dy1",
#              "harm", "beat", "measure", "key", "meter" ]
# abc reduce
df.columns = ["voice4", "dy4", "voice3", "dy3", "voice2", "dy2", "voice1", "dy1",
              "harm", "index", "beat", "measure", "key", "meter" ]

In [10]:
# process dataframe, delete bars, score representations, and "rest slice"
df = df[~df['beat'].astype(str).str.startswith(('=','.','*'))]
df

,voice4,dy4,voice3,dy3,voice2,dy2,voice1,dy1,harm,index,beat,measure,key,meter
2,8.G#,.,2r,.,2r,.,2r,.,I,10,1,=1,G#,M2/4
4,8.F##,.,2r,.,2r,.,2r,.,I,12,2,=1,G#,M2/4
7,8.E#,.,2r,.,2r,.,2r,.,I,15,1,=2,G#,M2/4
9,8.D#,.,2r,.,2r,.,2r,.,I,17,2,=2,G#,M2/4
12,8.C#,.,2r,.,2r,.,2r,.,I,20,1,=3,G#,M2/4
14,8BB#'L,.,2r,.,2r,.,2r,.,I,22,2,=3,G#,M2/4
17,2D#,.,2r,.,2r,.,2r,.,V,25,1,=4,G#,M2/4
19,4r,p,4r,p p,(8g#L,p,(4g#,.,I,28,1,=5,G#,M2/4
20,4r,p,4r,p p,8d#J,p,(4g#,.,I,29,1.5,=5,G#,M2/4
21,[4G#,p,(8f#L,p p,8g#L,p,[4b#),.,V7/IV,30,2,=5,G#,M2/4


In [11]:
prev_note_list = []
for index, row in df.iterrows():
    # for this onset slice 
    this_onset_vec = []
    
    ######################################### Process notes and onsets ########################################
    voices = ["voice4","voice3","voice2","voice1"]
    cur_note_tmp = []
    for v in voices:
        this_note = ''.join(row[[v]].values)
        if ' ' in this_note: #for a part has multiple voices
            this_note_list = this_note.split(" ")
            cur_note_tmp.extend(this_note_list)
        else:
            cur_note_tmp.append(this_note)
    
    cur_note_list = []
    for n in cur_note_tmp:
        note_name = re.sub('[^a-gA-G#-]+', '', n)
        if note_name:
            cur_note_list.append(note_name)
            
    cur_onset_list = list(set(cur_note_list) - set(prev_note_list))
    prev_note_list = cur_note_list
    
    
    note_vec = to_21d_vec(cur_note_list)
    onset_vec = to_21d_vec(cur_onset_list)
    this_onset_vec.extend(note_vec)
    this_onset_vec.extend(onset_vec)
    
    ######################################### Process beat position ########################################
    beat_pos = ''.join(row[["beat"]].values)
    meter = ''.join(row[["meter"]].values)
    meter = meter.replace("M", "")
    beat_vec = get_beat_vector(beat_pos, meter)
    this_onset_vec.extend(beat_vec)
    
    ######################################### Process chord label ########################################
    harm = ''.join(row[["harm"]].values)
    harm = re.sub('[()]', '', harm) # process string
    harm = specialChords(harm)
    harm = specialChordsABC(harm)
    key = ''.join(row[["key"]].values)
    key_harm = key + ":" + harm
    invalid = set()
    #print(key_harm)
    try:
        chord_label = inputRN(key_harm)["Chord label"]
    except:
        invalid.add(key_harm)
        continue # pass this onset slice if RN is not recognizable
    this_onset_vec.append(str(chord_label))
    #print(this_onset_vec)
  
    #########################################################################################################


Invalid entry. Try again. G#:V43b9
Invalid entry. Try again. G#:V43b9
Invalid entry. Try again. G#:V43b9
Invalid entry. Try again. G#:V43b9
Invalid entry. Try again. G#:V652/ii
Invalid entry. Try again. G#:V652/ii
Invalid entry. Try again. G#:V43b2
Invalid entry. Try again. G#:V43b2
Invalid entry. Try again. G#:I+b7+b2
Invalid entry. Try again. G#:iv64+6+#4
Invalid entry. Try again. G#:I+b7+b2
Invalid entry. Try again. G#:iv64+6+#4
Invalid entry. Try again. G#:@none
Invalid entry. Try again. G#:@none
Invalid entry. Try again. G#:@none
Invalid entry. Try again. G#:@none
Invalid entry. Try again. G#:@none
Invalid entry. Try again. G#:@none
Invalid entry. Try again. G#:@none
Invalid entry. Try again. G#:@none
Invalid entry. Try again. G#:@none
Invalid entry. Try again. G#:@none
Invalid entry. Try again. G#:#viio62/V
Invalid entry. Try again. G#:#viio62/V
Invalid entry. Try again. G#:#viio62/V
Invalid entry. Try again. G#:ii%65/iv
Invalid entry. Try again. G#:Vb9/iv
Invalid entry. Try agai

In [46]:
print(inputRN ("C:V+M7"))

{'Current key': 'C major', 'Tonicized key': 'None', 'Intervallic construction': 'GM3A5M7', 'Inversion': '0', 'Chord label': 'G augmented major seventh', 'Default function': 'dominant', 'Chord pitches': "('G', 'B', 'D#', 'F#')", 'Chord pitch classes': '(7, 11, 3, 6)'}
